# Modelling

#### Imports

In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from keras.models import load_model


from utils.utils import create_daily_sequences, test_sequences, get_optimization_results, plot_validation_loss, print_metrics, quick_result_plot, calculate_nrmse

#### Load data

In [ ]:
with open('Data/heatpump/data_heatpump_cleaned_v1.pkl', 'rb') as f:
    load_dict = pickle.load(f)

with open('Data/weather/data_weather_v1.pkl', 'rb') as f:
    weather_data = pickle.load(f)

building_info = pd.read_excel("Data/Gebaeudeinformationen_cleaned.xlsx", index_col=0)
building_info.set_index("Building number", inplace=True)

# add building information
for house in load_dict:
    id = int(re.findall(r'\d+', house)[0])

    load_dict[house]["area"] = building_info.loc[id]["Building area"]
    load_dict[house]["inhabitants"] = building_info.loc[id]["Number of inhabitants"]
    load_dict[house]["building"] = id
    
    weather_data = weather_data[weather_data.index>=1528965900]
    load_dict[house] = pd.concat([load_dict[house], weather_data], axis=1)

Globale Variablen

In [ ]:
SEQUENZE_LENGTH = 3  # in Tagen
PREDICTION_LENGTH = 1 # in Tagen

### Prepare Data for Modelling

In [ ]:
# Initialisierung des Scalers
scaler = MinMaxScaler()

# Listen für X und y arrays
all_X = []
all_y = []

for building_id in tqdm(load_dict, desc="Verarbeite Gebäude"):
    # Skalieren der Werte
    load_dict[building_id] = pd.DataFrame(scaler.fit_transform(load_dict[building_id]), columns=load_dict[building_id].columns)
    # Erstellen der Sequenzen
    X_building, y_building = create_daily_sequences(load_dict[building_id], SEQUENZE_LENGTH, PREDICTION_LENGTH)
    # Hinzufügen der Sequenzen zur Gesamtliste
    all_X.append(X_building)
    all_y.append(y_building)

# Für Tages Sequenzen
X = np.concatenate(all_X, axis=0)
y = np.concatenate(all_y, axis=0)

print("Dimensionen X: " + str(X.shape))
print("Dimensionen y: " + str(y.shape))

In [ ]:
test_sequences(X, y, SEQUENZE_LENGTH*96, PREDICTION_LENGTH*96)

In [ ]:
# Aufteilung in Trainings-, Test- und Validierungsdaten
# Ansatz: Chronologische Aufteilung ohne Überlappung
# sonst: sklearn.train_test_split() mit zufälliger Aufteilung der Daten -> Problem: Vorhersage der Vergangenheit mit Werten aus der Zukunft?

train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.85)

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:val_size], y[train_size:val_size]
X_test, y_test = X[val_size:], y[val_size:]

df_test_start = pd.DataFrame()
df_test_start['P_TOT'] = y_test[:, :, 0].flatten()
df_test_start['PF_TOT'] = y_test[:, :, 1].flatten()

y_train = y_train.reshape(-1, 96 * 2)   # Umformen in [Anzahl der Beispiele, 192]
y_val= y_val.reshape(-1, 96 * 2)        # Umformen in [Anzahl der Beispiele, 192]
y_test = y_test.reshape(-1, 96 * 2)     # Umformen in [Anzahl der Beispiele, 192]

### Modellarchitektur

Hyperparameter

In [ ]:
units_0 = 192
num_gru_layers = 4
hidden_units = [128, 96, 256, 160]
units_outputs = 96 * 2
dropout_rate_0 = 0.3
dropout_hidden = [0.3, 0.3, 0.2, 0.2]
batch_size = 32

In [ ]:
model = Sequential()

# Eingabeschicht
model.add(GRU(units=units_0,            # Hyperparameter -> kann variiert und angepasst werden
              return_sequences=True,    # Konfigurationsparameter, default: False
                                        # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                        #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                        #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                        #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                        # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                        #            ansonsten: return_sequences=False.
                                        # 
              input_shape=(
                  X.shape[1],           # Sequenzlänge
                  X.shape[2]            # Anzahl der Features
                )
            )
        )
# Dropout-Schicht
model.add(Dropout(dropout_rate_0))  # Dropout zur Vermeidung von Overfitting durch zufälliges Deaktivieren von Neuronen während des Trainingsprozesses -> Vermeidung dominanter Neuronen -> bessere Generalisierung

# Weitere GRU-Schichten
for i in range(0, num_gru_layers):
    model.add(GRU(units=hidden_units[i],
                  return_sequences= i < (num_gru_layers - 1)    # True für Schichten 1-3, False für Schicht 4
                )
            )
    model.add(Dropout(dropout_hidden[i]))

# Ausgabeschicht
model.add(Dense(units_outputs, activation='linear'))

In [ ]:
# Visualisierung der Modellstruktur
model.summary()
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
model.compile(
    optimizer='adam',                   # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                        # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                        # Adam - adaptive moment estimation: Grundprinzipien
                                        #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                        #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                        #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                        #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                        #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                        # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
    loss='mean_squared_error',          # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
    metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
    )

In [ ]:
# Trainieren des Modells
history = model.fit(
    X_train, y_train,                   # Übergabe der Trainingsdaten
    epochs=100,                         # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                        #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
    batch_size=batch_size,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                        #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
    validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
    use_multiprocessing=True,           # Laufzeitoptimierung
    workers=4,                          # Nutzen mehrerer CPU-Kerne
    verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
)

In [ ]:
model.save('models/GRU_4layer_1.h5')  # Speichert das Modell im HDF5-Format

In [ ]:
plot_validation_loss(history, "- Model 1")

Evaluierung des Modells mit den Testdaten

In [ ]:
model = load_model('models/GRU_4layer_1.h5')

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print('Testverlust:', test_loss)

In [ ]:
# Vorhersagen auf den Testdaten machen
predicted_test = model.predict(X_test)
print("Shape predicted_test {}".format(predicted_test.shape))
print("Shape y_test {}".format(y_test.shape))

In [ ]:
print_metrics(y_test, predicted_test)

In [ ]:
n = 0
print_metrics(y_test[n], predicted_test[n])
print("nRMSE: " + str(calculate_nrmse(y_test[n], predicted_test[n])))
quick_result_plot(predicted_test[n], y_test[n])

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

mape_dict = {}
nrsme_dict = {}
for i in range(1, len(X_test)): 
    mape = mean_absolute_percentage_error(y_test[i-1:i], predicted_test[i-1:i])
    nrsme = calculate_nrmse(y_test[i-1:i], predicted_test[i-1:i])
    mape_dict[i] = mape
    nrsme_dict[i] = nrsme

results = pd.Series(nrsme_dict).to_frame().rename(columns={0:"nRSME"})
results["MAPE"] = pd.Series(mape_dict)
results["nRSME"].describe()

In [ ]:
results[results["MAPE"]>100]

Rückskalierung der Daten

In [ ]:
# Reshape y_test zurück in die ursprüngliche Form [Anzahl der Beispiele, 96 Zeitpunkte, 2 Variablen]
predicted_test_reshaped = predicted_test.reshape(-1, 96, 2)
y_test_reshaped = y_test.reshape(-1, 96, 2)

# Erstellen eines leeren DataFrames für die Ergebnisse
df_pred = pd.DataFrame()
df_test = pd.DataFrame()

# Extrahieren und zuordnen der Daten zu den entsprechenden Spalten
df_pred['P_TOT_pred'] = predicted_test_reshaped[:, :, 0].flatten()
df_pred['PF_TOT_pred'] = predicted_test_reshaped[:, :, 1].flatten()
df_test['P_TOT'] = y_test_reshaped[:, :, 0].flatten()
df_test['PF_TOT'] = y_test_reshaped[:, :, 1].flatten()

# Erstellen eines temporären DataFrame mit der gleichen Struktur wie df_original_structure
temp_df = pd.DataFrame(0, index=np.arange(len(df_pred)), columns=load_dict["SFH10"].columns)
# Setzen der Werte für die Zielvariablen
temp_df['P_TOT'] = df_pred['P_TOT_pred']
temp_df['PF_TOT'] = df_pred['PF_TOT_pred']
# Anwenden von inverse_transform
temp_array = scaler.inverse_transform(temp_df)
# Erstellen eines neuen DataFrame mit den invers transformierten Werten
df_inverse_transformed = pd.DataFrame(temp_array, columns=load_dict["SFH10"].columns)
# Extrahieren der invers transformierten Zielvariablen
predictions = df_inverse_transformed[['P_TOT', 'PF_TOT']]
#predictions.index = df[int(len(df)*0.85):].index[:360864]

# Erstellen eines temporären DataFrame mit der gleichen Struktur wie df_original_structure
temp_df = pd.DataFrame(0, index=np.arange(len(df_test)), columns=load_dict["SFH10"].columns)
# Setzen der Werte für die Zielvariablen
temp_df['P_TOT'] = df_test['P_TOT']
temp_df['PF_TOT'] = df_test['PF_TOT']
# Anwenden von inverse_transform
temp_array = scaler.inverse_transform(temp_df)
# Erstellen eines neuen DataFrame mit den invers transformierten Werten
df_inverse_transformed = pd.DataFrame(temp_array, columns=load_dict["SFH10"].columns)
# Extrahieren der invers transformierten Zielvariablen
test_data = df_inverse_transformed[['P_TOT', 'PF_TOT']]
#test_data.index = df[int(len(df)*0.85):].index[:360864]

In [ ]:
print(calculate_nrmse(test_data["P_TOT"], predictions["P_TOT"]))
print(calculate_nrmse(test_data["PF_TOT"], predictions["PF_TOT"]))

In [ ]:
print(calculate_nrmse(y_test[:, 0:96], predicted_test[:, 0:96]))
print(calculate_nrmse(y_test[:, 96:], predicted_test[:, 96:]))

In [ ]:
print(mean_absolute_percentage_error(y_test[:, 0:96], predicted_test[:, 0:96]))
print(mean_absolute_percentage_error(test_data["P_TOT"], predictions["P_TOT"]))

In [ ]:
print(mean_absolute_percentage_error(y_test[:, 96:], predicted_test[:, 96:]))
print(mean_absolute_percentage_error(test_data["PF_TOT"], predictions["PF_TOT"]))

In [ ]:
df_test_start.equals(df_test)